In [63]:
with open("graph_data.pickle", 'rb') as f:
    data = pickle.load(f)

data

{'train-graph': Graph(num_nodes={'movie': 3706, 'user': 6040},
       num_edges={('movie', 'watched-by', 'user'): 1000209, ('user', 'watched', 'movie'): 1000209},
       metagraph=[('movie', 'user', 'watched-by'), ('user', 'movie', 'watched')]),
 'item-images': None,
 'user-type': 'user',
 'item-type': 'movie',
 'user-to-item-type': 'watched',
 'item-to-user-type': 'watched-by',
 'timestamp-edge-column': 'timestamp'}

In [164]:
g = data['train-graph']
user_type = data['user-type']
item_type = data['item-type']

In [165]:
g.number_of_nodes(item_type)

3706

In [166]:
user_to_item_etype =list(g.metagraph()[user_type][item_type])[0]
item_to_user_etype =list(g.metagraph()[item_type][user_type])[0]

In [167]:
import torch

In [168]:
a = torch.randint(0, g.number_of_nodes(item_type), (256,)) # 랜덤 아이템 노드를 뽑았고

In [169]:
import dgl

In [170]:
result = dgl.sampling.random_walk(g, a, metapath=[item_to_user_etype, user_to_item_etype])

In [171]:
mask = (result[0][:, 2] != -1)
len(mask)

256

In [173]:
random_walk_length=2
random_walk_restart_prob = 0.5
num_random_walks = 10
num_neighbors = 3

In [174]:
dgl.sampling.PinSAGESampler(g, item_type, user_type, random_walk_length,
                                        random_walk_restart_prob, num_random_walks, num_neighbors)

In [176]:
import scipy

In [187]:
g = scipy.sparse.random(3000, 5000, 0.003)
# 3000 by 5000 의 sparse matrix. 0과 1사이의 값을 가지며, 0이 아닌 확률은 0.003인 베르누이 확률분포를 따른다.

# g는 인접행렬 역할을 하고.. nonzero 값을 뽑으면 edge가 된다고 볼 수 있다.
G = dgl.heterograph({
    ('A', 'AB', 'B'): g.nonzero(),
    ('B', 'BA', 'A'): g.T.nonzero()})

G

Graph(num_nodes={'A': 3000, 'B': 5000},
      num_edges={('A', 'AB', 'B'): 45000, ('B', 'BA', 'A'): 45000},
      metagraph=[('A', 'B', 'AB'), ('B', 'A', 'BA')])

In [188]:
g.nonzero()

(array([ 606, 2043,  397, ...,  976, 1086, 2745]),
 array([2884,   82, 1197, ...,  952, 2011, 4863], dtype=int32))

In [185]:
# 'A'라는 노드타입으로 이루어진 호모지니어스한 그래프를 만들어낼 것.
# 각 노드는 최대 10개의 이웃을 가지도록 한다.
sampler = dgl.sampling.PinSAGESampler(G, 'A', 'B', 3, 0.5, 200, 10)

In [205]:
# 0, 1, 2 번 'A' 노드의 이웃을 샘플링
seeds = torch.LongTensor([0, 1, 2, 3,4])
frontier = sampler(seeds)
frontier.all_edges(form='uv')

(tensor([   0, 1020,  519, 1623, 1349,  959, 1603,  689,   28, 2143,    1, 1625,
         1110, 1624, 1472,    3, 2648, 2645, 2080, 1801,    2,  497, 2986, 2367,
         2353, 1942, 1864,  311, 2805, 2766,    3, 2906, 2310, 1873,  567,   21,
         2648, 1991, 1338, 1009,    4, 1802,  238,  157, 1266,    3, 2585, 2072,
         1880, 1735]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4]))

In [3]:
import torch
import torch.nn as nn
import pickle

In [4]:
with open("graph_data.pickle", 'rb') as f:
    dataset = pickle.load(f)

{'train-graph': Graph(num_nodes={'movie': 3706, 'user': 6040},
       num_edges={('movie', 'watched-by', 'user'): 1000209, ('user', 'watched', 'movie'): 1000209},
       metagraph=[('movie', 'user', 'watched-by'), ('user', 'movie', 'watched')]),
 'item-images': None,
 'user-type': 'user',
 'item-type': 'movie',
 'user-to-item-type': 'watched',
 'item-to-user-type': 'watched-by',
 'timestamp-edge-column': 'timestamp'}

In [5]:
g = dataset['train-graph']  # dgl graph
user_ntype = dataset['user-type']  # 'user'
item_ntype = dataset['item-type']  # 'movie'
device = torch.device("cuda")

In [8]:
from sampler import ItemToItemBatchSampler, NeighborSampler, PinSAGECollator
from torch.utils.data import DataLoader

In [9]:
# sampling
neighbor_sampler = NeighborSampler(
        g, user_ntype, item_ntype, 2,
        0.5, 10, 3,
        2)  # user의 이웃 노드를 핀세이지 알고리즘으로 샘플링

collator = PinSAGECollator(neighbor_sampler, g, item_ntype)

dataloader_test = DataLoader(
    torch.arange(g.number_of_nodes(item_ntype)),
    batch_size=256,
    collate_fn=collator.collate_test,
    num_workers=0)

In [13]:
checkpoint = torch.load("model.pt")
model_state_dict = checkpoint['model']

In [14]:
from model import PinSAGEModel

In [15]:
model = PinSAGEModel(g, item_ntype, 100, 2).to(device)

In [16]:
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [21]:
model.eval()
with torch.no_grad():
    h_item_batches = []
    for blocks in dataloader_test:
        blocks = [block.to(device) for block in blocks]
        h_item_batches.append(model.get_representation(blocks))
    h_item = torch.cat(h_item_batches, 0)


C:\Users\PC\.conda\envs\pin\lib\site-packages\dgl\backend\pytorch\tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


이건 아이템 임베딩인 것 같고.

In [24]:
from scipy import spatial

In [42]:
# item 임베딩
h_item

tensor([[-0.0205,  0.1565, -0.4017,  ..., -0.4634,  0.2782,  0.5463],
        [ 0.0835,  0.4176, -0.0970,  ..., -0.1294, -0.0708,  0.7626],
        [-0.1577,  0.1498, -0.4123,  ..., -0.2008,  0.1950,  0.4759],
        ...,
        [ 0.0964,  0.0837,  0.0607,  ..., -0.1032,  0.0800,  0.1806],
        [ 0.0964,  0.0489,  0.0607,  ..., -0.1032,  0.1006,  0.1938],
        [-0.0273,  0.0806, -0.0971,  ..., -0.1966,  0.2641,  0.1684]],
       device='cuda:0')

In [20]:
h_item.shape

torch.Size([3706, 100])

In [23]:
g.number_of_nodes(item_ntype)

3706

In [26]:
tree = spatial.KDTree(h_item.tolist())
tree

In [29]:
import numpy as np
# 2D 데이터 생성
data = np.array([(2, 3), (5, 4), (9, 6), (4, 7), (8, 1), (7, 2)])

# KDTree 생성
kdtree = spatial.KDTree(data)

# 새로운 데이터 포인트 생성
new_point = np.array([(3, 5)])

# KDTree에서 가장 가까운 이웃 검색
distance, index = kdtree.query(new_point)

# 결과 출력
print("가장 가까운 이웃의 인덱스: ", index)
print("가장 가까운 이웃과의 거리: ", distance)
print("가장 가까운 이웃: ", data[index])

가장 가까운 이웃의 인덱스:  [0]
가장 가까운 이웃과의 거리:  [2.23606798]
가장 가까운 이웃:  [[2 3]]
